In [1]:
import gensim 
import pandas as pd
import numpy
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

/Users/swang/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
cleaneddoc = pd.read_pickle('cleaned_doc.txt')
dictionary = gensim.corpora.Dictionary(cleaneddoc)
corpus = [dictionary.doc2bow(text) for text in cleaneddoc]
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


In [8]:
new_texts=[]
for i, doc in enumerate(corpus_tfidf):
    text = []
    for j, (_,tfidf) in enumerate(doc):
        if tfidf >= 0.3:
            text.append(cleaneddoc[i][j])
    new_texts.append(text)
    

In [4]:
new_dict = gensim.corpora.Dictionary(new_texts)
vocab = new_dict.token2id.keys()

NameError: name 'new_texts' is not defined

In [3]:
model = gensim.models.Word2Vec(cleaneddoc, size=50, window=6, min_count=5, workers=1)

In [4]:
vocabulary = sorted(model.vocab)
emb_tuple = tuple([model[v] for v in vocabulary])
X = numpy.vstack(emb_tuple)

In [5]:
X_embedded = TSNE(n_components=2, init='pca', random_state=0).fit_transform(X)

In [7]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label

source = ColumnDataSource(data=dict(x=list(X_embedded[:, 0]),
                                    y=list(X_embedded[:, 1]),
                                    words= vocabulary))

p = figure(title='Word2Vec tSNE')
p.scatter(x='x', y='y', size=2, source=source)
#labels = LabelSet(x='x', y='y', text='words', level='glyph',x_offset=5, y_offset=5, source=source, render_mode='canvas')
#p.add_layout(labels)

show(p)